#### Imports

In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 200
pd.options.display.max_columns = 100
pd.set_option('display.float_format', lambda x: '%.3f' % x)
from math import floor
from sklearn.cluster import KMeans

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

#### Read Data

In [6]:
df = pd.read_csv("..\data\cs-training.csv", low_memory=False)

In [ ]:
del df['Unnamed: 0']

#### Sneak peak at data

In [9]:
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,0.766,45,2,0.803,9120.000,13,0,6,0,2.000
1,0,0.957,40,0,0.122,2600.000,4,0,0,0,1.000
2,0,0.658,38,1,0.085,3042.000,2,1,0,0,0.000
3,0,0.234,30,0,0.036,3300.000,5,0,0,0,0.000
4,0,0.907,49,1,0.025,63588.000,7,0,1,0,0.000


#### Target Variable

In [12]:
df.SeriousDlqin2yrs.value_counts()

0    139974
1     10026
Name: SeriousDlqin2yrs, dtype: int64

####  Basic stats of data

In [13]:
df.describe()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
count,150000.000,150000.000,150000.000,150000.000,150000.000,120269.000,150000.000,150000.000,150000.000,150000.000,146076.000
mean,0.067,6.048,52.295,0.421,353.005,6670.221,8.453,0.266,1.018,0.240,0.757
std,0.250,249.755,14.772,4.193,2037.819,14384.674,5.146,4.169,1.130,4.155,1.115
min,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,0.000,0.030,41.000,0.000,0.175,3400.000,5.000,0.000,0.000,0.000,0.000
50%,0.000,0.154,52.000,0.000,0.367,5400.000,8.000,0.000,1.000,0.000,0.000
75%,0.000,0.559,63.000,0.000,0.868,8249.000,11.000,0.000,2.000,0.000,1.000
max,1.000,50708.000,109.000,98.000,329664.000,3008750.000,58.000,98.000,54.000,98.000,20.000


#### Null Containing columns

In [14]:
df.isnull().sum()

SeriousDlqin2yrs                            0
RevolvingUtilizationOfUnsecuredLines        0
age                                         0
NumberOfTime30-59DaysPastDueNotWorse        0
DebtRatio                                   0
MonthlyIncome                           29731
NumberOfOpenCreditLinesAndLoans             0
NumberOfTimes90DaysLate                     0
NumberRealEstateLoansOrLines                0
NumberOfTime60-89DaysPastDueNotWorse        0
NumberOfDependents                       3924
dtype: int64

In [17]:
with_null_income = df[df.MonthlyIncome.isnull()]

In [21]:
with_null_income.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
6,0,0.306,57,0,5710.000,nan,8,0,3,0,0.000
8,0,0.117,27,0,46.000,nan,2,0,0,0,nan
16,0,0.061,78,0,2058.000,nan,10,0,2,0,0.000
32,0,0.083,62,0,977.000,nan,6,0,1,0,0.000
41,0,0.073,81,0,75.000,nan,7,0,0,0,0.000


In [19]:
with_null_income.SeriousDlqin2yrs.value_counts()

0    28062
1     1669
Name: SeriousDlqin2yrs, dtype: int64

In [22]:
with_null_income.describe()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
count,29731.000,29731.000,29731.000,29731.000,29731.000,0.000,29731.000,29731.000,29731.000,29731.000,25807.000
mean,0.056,6.649,56.362,0.580,1673.397,nan,7.216,0.485,0.871,0.453,0.316
std,0.230,217.815,15.439,6.255,4248.373,nan,4.843,6.250,1.034,6.242,0.810
min,0.000,0.000,21.000,0.000,0.000,nan,0.000,0.000,0.000,0.000,0.000
25%,0.000,0.016,46.000,0.000,123.000,nan,4.000,0.000,0.000,0.000,0.000
50%,0.000,0.082,57.000,0.000,1159.000,nan,6.000,0.000,1.000,0.000,0.000
75%,0.000,0.441,67.000,0.000,2382.000,nan,10.000,0.000,1.000,0.000,0.000
max,1.000,22198.000,109.000,98.000,329664.000,nan,45.000,98.000,23.000,98.000,9.000
